In [3]:
import pandas as pd
import json
import logging
import time
import backoff
import boto3
import pandas as pd


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 1000)

client = boto3.client("lambda")


df = pd.read_csv(
    filepath_or_buffer="../data/enriched/votos_por_secao_com_cep.csv",
    encoding="utf-8",
    header=0,
    dtype={
        0: int,
        1: int,
        2: str,
        3: str,
        4: str,
        5: str,
        6: str,
        7: int,
        8: int,
        9: str,
        10: int,
        },
    )
df["CEP"] = df["CEP"].str[:-4]

df = df[["CEP", "TURNO", "NUMERO_CANDIDATO", "VOTOS"]]
df = df.groupby(["CEP", "TURNO", "NUMERO_CANDIDATO"]).sum()
df = df.reset_index()
total_rows = df.shape[0]

@backoff.on_exception(backoff.expo, RuntimeError, max_time=15)
def call_lambda(payload):
    print(payload)
    response = client.invoke(
        FunctionName="Eleicoes2022-DataProcessi-GeoVotesFunctionCA5674F9-5OLJWl1KhVGr",
        InvocationType="Event",
        Payload=json.dumps(payload),
    )
    status_code = response['StatusCode']
    if status_code < 200 and status_code >= 300:
        raise RuntimeError

for row in df.itertuples():
    if row < 2294:
        continue
    print(f"Processing row {row.Index} of {total_rows}")
    payload = {"cep": row.CEP, "turno": row.TURNO, "numero_candidato": row.NUMERO_CANDIDATO, "votos": row.VOTOS}
    call_lambda(payload)
    time.sleep(row.VOTOS/300)

Processing row 0 of 90479
{'cep': '0100', 'turno': 1, 'numero_candidato': 12, 'votos': 4980}
Processing row 1 of 90479
{'cep': '0100', 'turno': 1, 'numero_candidato': 13, 'votos': 61416}
Processing row 2 of 90479
{'cep': '0100', 'turno': 1, 'numero_candidato': 14, 'votos': 51}
Processing row 3 of 90479
{'cep': '0100', 'turno': 1, 'numero_candidato': 15, 'votos': 8714}
Processing row 4 of 90479
{'cep': '0100', 'turno': 1, 'numero_candidato': 16, 'votos': 66}
Processing row 5 of 90479
{'cep': '0100', 'turno': 1, 'numero_candidato': 21, 'votos': 112}
Processing row 6 of 90479
{'cep': '0100', 'turno': 1, 'numero_candidato': 22, 'votos': 36985}
Processing row 7 of 90479
{'cep': '0100', 'turno': 1, 'numero_candidato': 27, 'votos': 26}
Processing row 8 of 90479
{'cep': '0100', 'turno': 1, 'numero_candidato': 30, 'votos': 1094}
Processing row 9 of 90479
{'cep': '0100', 'turno': 1, 'numero_candidato': 44, 'votos': 664}
Processing row 10 of 90479
{'cep': '0100', 'turno': 1, 'numero_candidato': 8

KeyboardInterrupt: 

In [7]:
import boto3

client = boto3.client("sqs")

url='https://sqs.us-east-1.amazonaws.com/317456194246/Eleicoes2022-DataProcessing-GeoVotesFunctionDeadLetterQueueE7A009DE-9Ei3JDPcrg0o'
messages = client.receive_message(QueueUrl=url, MaxNumberOfMessages=10)["Messages"]

for message in messages:
    call_lambda(message["Body"])
    print(message)
    client.delete_message(
        QueueUrl=url,
        ReceiptHandle=message["ReceiptHandle"]
    )


[{'MessageId': 'c1f0990e-2d13-49ea-91ce-bce72906b95e', 'ReceiptHandle': 'AQEB0Gn+XNZpxZSLOsmmAJ7RN6F2/0resp0ebsLVw9qytHbu6+Hm5PugeJ4Iqc7SpajaTcC/oOOPigB8/6eRhi95xUhjeGgOCU3bFqC+CaAVmhwmbwAld6s7FUl9h4H0eCkzDjsXAxXSqf6Hhzei1wI0ssGzIUNCfaz5EzdWXifXl3yyr8MCeK5Lc/4GfSwEJR4ndvGJRku2vhbVuvH3pYQHMctCejnVSkM5rMGJglfOJVFRY+AIRApvcxzJCzbKlS5Yw/F8Srk1wupkSH1dwqkiGq+5TuPoStHhmzoX620u+xKa8sCLi1p9nfYFqROhjMYLUn40Ys/Ld9g9ZS2I4xEDZnSP8BgxgTOW7UXQhaO3jmGaKwitpHxBaHCiRwHtH8EEa7d9KG+1mYvx8GibPOcn+r4+l0k/D9fsFum8r5ZeQvcat0RUgA3mMeTORR7e6eMjU1ZC4k25r181MFzJ5R/cakVAv6v29KQ35Uwf06qgo40=', 'MD5OfBody': '167e151a0b356646b4c5f32df219bdab', 'Body': '{"cep": "095", "numero_candidato": 12, "votos": 276}'}, {'MessageId': '23a350a1-99d3-42c5-b5dd-3e86e592ebcb', 'ReceiptHandle': 'AQEBtV/sPxPnqw6S8WqEgc5LAa087LtWBBrM9PmFxRo/uiGpdlKl9G+NyZWG7OIB2KrtEPQmVElmn+d4z+0+yazD+T7F3i7d/sppfK6R7comio7Xr+pWJ0u1h9BNdh4i4C3V/epLcYgByxd+LiLOKUhwrI/JeMmU9QLa+8lvsX2NinmGPwpgHVbox4HDFr3JpyfzVc8zwe41cDn0l1+bWLMcz+rq/JtUCnPaypQcNypKdZvLxq